In [1]:
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
#from keras.optimizers import Adam
import tensorflow as tf
#kb.set_session(kb.tf.Session(config=kb.tf.ConfigProto(intra_op_parallelism_threads=12, inter_op_parallelism_threads=12)))
#tf.config.threading.set_intra_op_parallelism_threads(12)
#tf.config.threading.set_inter_op_parallelism_threads(12)

import mdtraj as md
import numpy as np
import keras as krs

In [2]:
# too verbose
# tf.debugging.set_log_device_placement(True)

In [8]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
print(tf.config.get_logical_device_configuration(physical_devices[0]))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
None


In [30]:
refpdb = md.load_pdb("md_ala_final_noPBC_fit_noH.pdb")
X = md.load("md_ala_final_noPBC_fit_noH.xtc", top="md_ala_final_noPBC_fit_noH.pdb")
X.superpose(refpdb)
Xt = np.zeros((X.n_frames, X.n_atoms*3, 1))
for i in range(X.n_frames):
  for j in range(X.n_atoms):
    Xt[i,3*j,0] = X.xyz[i,j,0]
    Xt[i,3*j+1,0] = X.xyz[i,j,1]
    Xt[i,3*j+2,0] = X.xyz[i,j,2]
X_train = Xt
scale = [np.min(X_train), np.max(X_train)]
X_train = (X_train - scale[0])/(scale[1] - scale[0])

mdtraj/rmsd/src/theobald_rmsd.cpp UNCONVERGED ROTATION MATRIX. RETURNING IDENTITY=296
mdtraj/rmsd/src/theobald_rmsd.cpp UNCONVERGED ROTATION MATRIX. RETURNING IDENTITY=296
mdtraj/rmsd/src/theobald_rmsd.cpp UNCONVERGED ROTATION MATRIX. RETURNING IDENTITY=296
mdtraj/rmsd/src/theobald_rmsd.cpp UNCONVERGED ROTATION MATRIX. RETURNING IDENTITY=296
mdtraj/rmsd/src/theobald_rmsd.cpp UNCONVERGED ROTATION MATRIX. RETURNING IDENTITY=296
mdtraj/rmsd/src/theobald_rmsd.cpp UNCONVERGED ROTATION MATRIX. RETURNING IDENTITY=296
mdtraj/rmsd/src/theobald_rmsd.cpp UNCONVERGED ROTATION MATRIX. RETURNING IDENTITY=296


In [34]:
#dev='/GPU:0'
dev='/CPU:0'

In [35]:
with tf.device(dev):
    input_coord = krs.layers.Input(shape=(3*X.n_atoms,))
    encoded = krs.layers.Dense(32, activation='sigmoid', use_bias=True)(input_coord)
    encoded = krs.layers.Dense(32, activation='sigmoid', use_bias=True)(encoded)
    encoded = krs.layers.Dense(2, activation='linear', use_bias=True)(encoded)

    decoded = krs.layers.Dense(32, activation='sigmoid', use_bias=True)(encoded)
    decoded = krs.layers.Dense(32, activation='sigmoid', use_bias=True)(decoded)
    decoded = krs.layers.Dense(3*X.n_atoms, activation='linear', use_bias=True)(decoded)
    codecvs = krs.models.Model(input_coord, decoded)
    encoder = krs.models.Model(input_coord, encoded)

    codecvs.compile(optimizer='Adam', loss='mean_squared_error')

In [36]:
with tf.device(dev):
    history = codecvs.fit(X_train, X_train,
            epochs=5,
            batch_size=256,
            validation_data=(X_train, X_train))

Epoch 1/5
3907/3907 [==============================] - 6s 2ms/step - loss: 0.0041 - val_loss: 4.1571e-04
Epoch 2/5
3907/3907 [==============================] - 6s 2ms/step - loss: 2.5051e-04 - val_loss: 5.7953e-05
Epoch 3/5
3907/3907 [==============================] - 6s 2ms/step - loss: 5.6839e-05 - val_loss: 5.8149e-05
Epoch 4/5
3907/3907 [==============================] - 6s 1ms/step - loss: 5.6617e-05 - val_loss: 5.8004e-05
Epoch 5/5
3907/3907 [==============================] - 6s 1ms/step - loss: 5.5165e-05 - val_loss: 5.8445e-05


In [ ]:
coded_ala = encoder.predict(X_train)
np.savetxt("out.txt", coded_ala)
np.savetxt("loss.txt", history.history['loss'])

In [ ]:
decoded_ala = codecvs.predict(X_train)
decoded_ala = decoded_ala*(scale[1] - scale[0]) + scale[0]
np.savetxt("outc.txt", decoded_ala)